# Reglerentwurf

In [ ]:
# system imports
import sys

# 3rd party imports
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from scipy.optimize import curve_fit
from scipy.integrate import solve_ivp

# user imports
import readRst

sys.path.append('../')

We want to implement a control for the tank system. For this we have different approaches to model the tank system:
1. physical modeling
2. PT2 element from smoothed step response of the lower tank
3. series connection of two PT1 elements, whose parameters are obtained from two different step responses
4. Reinisch and Ziegler Nichols

TODO:
Bild mit Struktur, wie soll gesamtsystem aussehen, ein bisschen theorie

__Aufbau auswählen__

In [ ]:
aufbau = 1

## Verfahren

- Kurze Übersicht über die Verfahren
- Warum schwingen vermeiden -> kurze erläuterung
- welche verfahren bleiben

### 4. Reinisch and Ziegler Nichols

In [ ]:
# dataloading
data = pd.read_csv('../../../Messungen/2Tanksystem/20221011_Schwingung_Digitalregler_Aufbau1_V12_2_0_vU_V21_2_1vU.csv')

# display results
plt.close()

# creating the subplots
fig, axes = plt.subplots(2, 1, sharex=True, figsize=(8,7))

# plotting the values
data.plot('time', 'HeightT2', ax=axes[0])
data.plot('time', 'CtrlOutput', ax=axes[1])

# grid on for all axes
for ax in axes:
    ax.grid()

# setting x and y label
axes[1].set_xlabel('time [s]')
axes[0].set_ylabel('Height [m]')
axes[1].set_ylabel('Voltage [V]')

plt.show()

Unfortunately, harmonic oscillation is not possible due to regulation value limitation. Accordingly, this method is discarded.

### 5. Comparison

In [ ]:
dp = 5 # decimal places
d = {   'KP': [round(Kp_analytic,dp), round(Kp_pt2,dp), round(Kp_pt1,dp)],
        'T1': [round(T1_analytic,dp), round(T1_pt2,dp), round(T1_pt1,dp)],
        'T2': [round(T2_analytic,dp), round(T2_pt2,dp), round(T2_pt1,dp)],}
print ("{:<5} {:<11} {:<9} {:<11}".format(' ','modelling','PT2','PT1'))
for name, values in d.items():
    s1, s2, s3 = values
    print ("{:<5} {:<11} {:<9} {:<11}".format(name, s1, s2, s3))

### 6. Control Parameters

Symmetric optimum was rejected because there is no dominant time constant. The criterion $T_1 \gg a^2 \cdot T_2$ does not hold.

#### optimum criteria (page 37 "EinführungRegelungstechnik" PDF)

In [ ]:
# with modelled parameters
Kp_sym_analytic = T1_analytic / (2*Kp_analytic*T2_analytic)
Ki_sym_analytic = Kp_sym_analytic / T1_analytic

# with parameters of pt2 measurement
Kp_sym_pt2 = T1_pt2 / (2*Kp_pt2*T2_pt2)
Ki_sym_pt2 = Kp_sym_pt2 / T1_pt2

# with parameters of pt1 measurements
Kp_sym_pt1 = T1_pt1 / (2*Kp_pt1*T2_pt1)
Ki_sym_pt1 = Kp_sym_pt1 / T1_pt1

# print results
dp = 5 # decimal places
d = {   'KP': [round(Kp_sym_analytic,dp), round(Kp_sym_pt2,dp), round(Kp_sym_pt1,dp)],
        'KI': [round(Ki_sym_analytic,dp), round(Ki_sym_pt2,dp), round(Ki_sym_pt1,dp)],}
print ("{:<5} {:<11} {:<9} {:<11}".format(' ','modelling','PT2','PT1'))
for name, values in d.items():
    s1, s2, s3 = values
    print ("{:<5} {:<11} {:<9} {:<11}".format(name, s1, s2, s3))

# Compare models